# Section Detection Using Medspacy

This example demonstreates how find sections based on regex pattern (not medspacy related) and dynamically build sections from results.

## Part 1 - Load medspacy dependencies.
Install python dependencies

In [ ]:
!python -m pip install medspacy

In [460]:
import spacy
from medspacy.section_detection import Sectionizer, SectionRule

nlp = medspacy.load()
sectionizer = nlp.add_pipe("medspacy_sectionizer", config={"rules":None})

help(Sectionizer)

Help on class Sectionizer in module medspacy.section_detection.sectionizer:

class Sectionizer(builtins.object)
 |  Sectionizer(nlp: spacy.language.Language, name: str = 'medspacy_sectionizer', rules: Optional[str] = 'default', max_section_length: Optional[int] = None, phrase_matcher_attr: str = 'LOWER', require_start_line: bool = False, require_end_line: bool = False, newline_pattern: str = '[\\n\\r]+[\\s]*$', input_span_type: Optional[Literal['ents', 'group']] = 'ents', span_group_name: str = 'medspacy_spans', span_attrs: Union[Literal['default'], Dict[str, Dict[str, Any]], NoneType] = 'default')
 |  
 |  The Sectionizer will search for spans in the text which match section header rules, such as 'Past Medical History:'.
 |  Sections will be represented in custom attributes as:
 |      category: A normalized title of the section. Example: 'past_medical_history'
 |      section_title: The Span of the doc which was matched as a section header.
 |          Example: 'Past Medical History:

## Part 2 - Define Medical Document & Identify Sections

In this experiment, regex patterns are used to identify new sections. The default medspacy sectionizer does not pickup all of the sections. Some pre-processing is done to clean up the section names since they will be used in a for loop to create new section rules.

In [461]:
medical_doc = """REASON FOR REFERRAL: , Ms. A is a 60-year-old African-American female with 12 years of education who was referred for neuropsychological evaluation by Dr. X after she demonstrated mild cognitive deficits on a neuropsychological screening evaluation during a followup appointment with him for stroke in July.  A comprehensive evaluation was requested to assess current cognitive functioning and assist with diagnostic decisions and treatment planning.,RELEVANT BACKGROUND INFORMATION:,  Historical information was obtained from a review of available medical records and clinical interview with Ms. A.  A summary of pertinent information is presented below.  Please refer to the patient's medical chart for a more complete history.,HISTORY OF PRESENTING PROBLEM:,  Ms. A presented to the ABC Hospital emergency department on 07/26/2009 reporting that after she had woken up that morning she noticed numbness and weakness in her left hand, slurred speech, and left facial droop.  Neurological evaluation with Dr. X confirmed left hemiparesis.  Brain CT showed no evidence of intracranial hemorrhage or mass effect and that she received TPA and had moderate improvement in left-sided weakness.  These symptoms were thought to be due to a right middle cerebral artery stroke.  She was transferred to the ICU for monitoring.  Ultrasound of the carotids showed 20% to 30% stenosis of the right ICA and 0% to 19% stenosis of the left ICA.  On 07/29/2009, she was admitted for acute inpatient rehabilitation for the treatment of residual functional deficits of her acute ischemic right MCA/CVA.  At discharge on 08/06/2009, she was mainly on supervision for all ADLs and walking with a rolling walker, but tolerating increased ambulation with a cane.  She was discharged home with recommendations for outpatient physical therapy.  She returned to the Sinai ER on 08/2009/2009 due to reported left arm pain, numbness, and weakness, which lasted 10 to 15 minutes and she reported that it felt "just like the stroke."  Brain CT on 08/2009/2009 was read as showing "mild chronic microvascular ischemic change of deep white matter," but no acute or significant interval change compared to her previous scan.  Neurological examination with Dr. Y was within normal limits, but she was admitted for a more extensive workup.  Due to left arm pain an ultrasound was completed on her left upper extremity, but it did not show deep vein thrombosis.,Followup CT on 08/10/2009 showed no significant interval change.  MRI could not be completed due to the patient's weight.  She was discharged on 08/11/2009 in stable condition after it was determined that this event was not neurological in origin; however, note that Ms. A referred to this as a second stroke.,Ms. A presented for a followup outpatient neurological evaluation with Dr. X on 09/22/2009, at which time a brief neuropsychological screening was also conducted.  She demonstrated significant impairments in confrontation naming, abstract verbal reasoning, and visual and verbal memory and thus a more comprehensive evaluation was suggested due to her intent to return to her full-time work duty.  During the current interview, Ms. A reported that she noticed mild memory problems including some difficultly remembering conversations, events, and at times forgetting to take her medications.  She also reported mild difficulty finding words in conversation, solving novel problems and tasks (e.g. difficulty learning to use her camcorder), but overall denied significant cognitive deficits in attention, concentration, language or other areas of cognitive functioning.  When asked about her return to work, she said that she was still on light duty due to limited physical activity because of residual left leg weakness.  She reported that no one had indicated to her that she appeared less capable of performing her job duties, but said that she was also receiving fewer files to process and enter data into the computer at the Social Security Agency that she works at.  Note also that she had some difficulty explaining exactly what her job involved.  She also reported having problems falling asleep at work and that she is working full-time although on light duty.,OTHER MEDICAL HISTORY:  ,As mentioned, Ms. A continues to have some residual left leg weakness and continues to use a rolling walker for ambulation, but she reported that her motor functioning had improved significantly.  She was diagnosed with sleep apnea approximately two years ago and was recently counseled by Dr. X on the need to use her CPAP because she indicated she never used it at night.  She reported that since her appointment with Dr. X, she has been using it "every other night."  When asked about daytime fatigue, Ms. A initially denied that she was having any difficulties, but repeatedly indicated that she was falling asleep at work and thought that it was due to looking at a computer screen.  She reported at times "snoring" and forgetting where she is at and said that a supervisor offered to give her coffee at one point.  She receives approximately two to five hours of sleep per night.  Other current untreated risk factors include obesity and hypercholesterolemia.  Her medical history is also significant for hypertension, asthma, abdominal adenocarcinoma status post hysterectomy with bilateral salpingo-oophorectomy, colonic benign polyps status post resection, benign lesions of the breast status post lumpectomy, and deep vein thrombosis in the left lower extremity status post six months of anticoagulation (which she had discontinued just prior to her stroke).,CURRENT MEDICATIONS: , Aspirin 81 mg daily, Colace 100 mg b.i.d., Lipitor 80 mg daily, and albuterol MDI p.r.n.,SUBSTANCE USE:,  Ms. A denied drinking alcohol or using illicit drugs.  She used to smoke a pack of cigarettes per day, but quit five to six years ago.,FAMILY MEDICAL HISTORY: , Ms. A had difficulty providing information on familial medical history.  She reported that her mother died three to four years ago from lung cancer.  Her father has gout and blood clots.  Siblings have reportedly been treated for asthma and GI tumors.  She was unsure of familial history of other conditions such as hypertension, high cholesterol, stroke, etc.,SOCIAL HISTORY: , Ms. A completed high school degree.  She reported that she primarily obtained B's and C's in school.  She received some tutoring for algebra in middle school, but denied ever having been held back a grade failing any classes or having any problems with attention or hyperactivity.,She currently works for the Social Security Administration in data processing.  As mentioned, she has returned to full-time work, but continues to perform only light duties due to her physical condition.  She is now living on her own.  She has never driven.  She reported that she continues to perform ADLs independently such as cooking and cleaning.  She lost her husband in 2005 and has three adult daughters.  She previously reported some concerns that her children wanted her to move into assisted living, but she did not discuss that during this current evaluation.  She also reported number of other family members who had recently passed away.  She has returned to activities she enjoys such as quire, knitting, and cooking and plans to go on a cruise to the Bahamas at the end of October.,PSYCHIATRIC HISTORY: , Ms. A did not report a history of psychological or psychiatric treatment.  She reported that her current mood was good, but did describe some anxiety and nervousness about various issues such as her return to work, her upcoming trip, and other events.  She reported that this only "comes and goes.",TASKS ADMINISTERED:,Clinical Interview,Adult History Questionnaire,Wechsler Test of Adult Reading (WTAR),Mini Mental Status Exam (MMSE),Cognistat Neurobehavioral Cognitive Status Examination,Repeatable Battery for the Assessment of Neuropsychological Status (RBANS; Form XX),Mattis Dementia Rating Scale, 2nd Edition (DRS-2),Neuropsychological Assessment Battery (NAB),Wechsler Adult Intelligence Scale, Third Edition (WAIS-III),Wechsler Adult Intelligence Scale, Fourth Edition (WAIS-IV),Wechsler Abbreviated Scale of Intelligence (WASI),Test of Variables of Attention (TOVA),Auditory Consonant Trigrams (ACT),Paced Auditory Serial Addition Test (PASAT),Ruff 2 & 7 Selective Attention Test,Symbol Digit Modalities Test (SDMT),Multilingual Aphasia Examination, Second Edition (MAE-II),  Token Test,  Sentence Repetition,  Visual Naming,  Controlled Oral Word Association,  Spelling Test,  Aural Comprehension,  Reading Comprehension,Boston Naming Test, Second Edition (BNT-2),Animal Naming Test"""

In [462]:
import re

# Find new sections using regex patterns & clean up strings
sections = [x.replace(":","").replace(",","") for x in re.findall('[A-Z, ]*:', medical_doc)]
sections

['REASON FOR REFERRAL',
 'RELEVANT BACKGROUND INFORMATION',
 'HISTORY OF PRESENTING PROBLEM',
 'OTHER MEDICAL HISTORY',
 'CURRENT MEDICATIONS',
 'SUBSTANCE USE',
 'FAMILY MEDICAL HISTORY',
 'SOCIAL HISTORY',
 'PSYCHIATRIC HISTORY',
 'TASKS ADMINISTERED']

## Part 3 - Create Section Rules From Sections List

In [463]:
new_patterns = []

for s in sections:
    section_cat = s.lower().replace(" ","_").replace(":","")
    section_lit = "{}:".format(s)

    new_patterns.append(SectionRule(category=section_cat, literal=section_lit))

sectionizer.add(new_patterns)

sectionizer.rules


[SectionRule(literal="REASON FOR REFERRAL:", category="reason_for_referral", pattern=None, on_match=None, parents=None, parent_required=False),
 SectionRule(literal="RELEVANT BACKGROUND INFORMATION:", category="relevant_background_information", pattern=None, on_match=None, parents=None, parent_required=False),
 SectionRule(literal="HISTORY OF PRESENTING PROBLEM:", category="history_of_presenting_problem", pattern=None, on_match=None, parents=None, parent_required=False),
 SectionRule(literal="OTHER MEDICAL HISTORY:", category="other_medical_history", pattern=None, on_match=None, parents=None, parent_required=False),
 SectionRule(literal="CURRENT MEDICATIONS:", category="current_medications", pattern=None, on_match=None, parents=None, parent_required=False),
 SectionRule(literal="SUBSTANCE USE:", category="substance_use", pattern=None, on_match=None, parents=None, parent_required=False),
 SectionRule(literal="FAMILY MEDICAL HISTORY:", category="family_medical_history", pattern=None, on_

In [464]:
doc = nlp(medical_doc)
visualize_ent(doc)

In [465]:
doc._.section_titles

[REASON FOR REFERRAL:,
 RELEVANT BACKGROUND INFORMATION:,
 HISTORY OF PRESENTING PROBLEM:,
 OTHER MEDICAL HISTORY:,
 CURRENT MEDICATIONS:,
 SUBSTANCE USE:,
 FAMILY MEDICAL HISTORY:,
 SOCIAL HISTORY:,
 PSYCHIATRIC HISTORY:,
 TASKS ADMINISTERED:]

In [466]:
#source https://github.com/medspacy/medspacy/blob/master/notebooks/section_detection/3-Subsections.ipynb
#additional post-processing added for this particular scenario (line 7)

for section in doc._.sections:
    title_span = doc[section.title_span[0] : section.title_span[1]]
    body_span = doc[section.body_span[0] : section.body_span[1]]
    body_span = re.sub(r"^([, ]*\b)", "", body_span.text) # post process body_span. this could probably be moved to pre-processing.
    
    print("CATEGORY.............. {0}".format(section.category))
    print("TITLE................. {0}".format(title_span))
    if section.parent:
        print("PARENT................ {0}".format(section.parent.category))
    else:
        print("PARENT................ {0}".format(section.parent))
    print("SECTION TEXT..........\n{0}".format(body_span))
    print("----------------------")

CATEGORY.............. reason_for_referral
TITLE................. REASON FOR REFERRAL:
PARENT................ None
SECTION TEXT..........
Ms. A is a 60-year-old African-American female with 12 years of education who was referred for neuropsychological evaluation by Dr. X after she demonstrated mild cognitive deficits on a neuropsychological screening evaluation during a followup appointment with him for stroke in July.  A comprehensive evaluation was requested to assess current cognitive functioning and assist with diagnostic decisions and treatment planning.,
----------------------
CATEGORY.............. relevant_background_information
TITLE................. RELEVANT BACKGROUND INFORMATION:
PARENT................ None
SECTION TEXT..........
Historical information was obtained from a review of available medical records and clinical interview with Ms. A.  A summary of pertinent information is presented below.  Please refer to the patient's medical chart for a more complete history.,
---